#  Description

Data Source: [Kaggle](https://www.kaggle.com/datasets/robikscube/hourly-energy-consumption?select=PJMW_hourly.csv)

### EnerNed Hourly Energy Consumption Data Scenario

EnerNed is a regional transmission organization in the United States. It is part of the Eastern Interconnection grid operating an electric transmission system in various states.

The company noticed that sometimes it is hard for them to assume what the future demand for power will be. They would like to have a predictive model that would help them in being ready for the increase in demand as well as regulating the transmission in the system once the demand is lower. They would like to also understand how the trends are shaped over time.

The hourly power consumption data comes from EnerNed's website and are in megawatts (MW). Follow the steps below to create a time series model. Please fill the code in the cells where indicated. Don’t worry about the time. If you don’t manage to finish all the exercises, you can always do it at home and compare your answers with the solutions provided in our repository.

Good luck!


# Dependencies

In [ ]:
# run just once and then comment it out
%cd ..
%conda env create -f environment.yml

In [ ]:
%conda activate pyladies

In [1]:
import pandas as pd
from pycaret.regression import *
from sklearn.model_selection import TimeSeriesSplit
import xgboost as xgb
import matplotlib.pyplot as plt


# Read Raw Data

Our first step is loading the data. Then we are going to do some initial operations like changing the datatypes, sorting it and setting the index.

In [4]:
raw_data = pd.read_csv("PJMW_hourly.csv").copy()

In [ ]:
# Show top 10 rows of th dataset
raw_data.

Code below saves names of the date and target columns. It will facilitate the next steps.

In [6]:
date_col = "Datetime"
target = "PJMW_MW"

Transform columns to appropriate data types. 
Use the following [link](https://www.geeksforgeeks.org/change-data-type-for-one-or-more-columns-in-pandas-dataframe/) in case you are stuck.

In [ ]:
# Make the date column datetime type 
raw_data[date_col] =

# Make the target column "float32"
raw_data[target] = 

Sort Dataframe and set index. Use the following [link](https://www.geeksforgeeks.org/how-to-sort-pandas-dataframe/) & [link](https://www.geeksforgeeks.org/python-pandas-dataframe-set_index/) in case you are stuck.

In [ ]:
# Sort values by the date column
raw_data = raw_data.

# Set the date columns as an index
raw_data = raw_data.

# Feature Engineering

__ALWAYS THINK OF THE PROBLEM BEFORE CREATING FEATURES!__

In this case, we need to predict hourly consumption of energy for 1 year ahead. Therefore, we will not be able to create features such as lag_1_day, lag_2_months and so forth. Instead, we must create features such as lag_1_year, lag_14_months and so forth. Always think of data availability. You cannot create lag/rolling features in the future, except for those that go back at least for the same time frame as the forecasting horizon, in this case 1 year.

Something that can be done is to incorporate other features such as GDP, PCI and so forth, but when we are forecasting out of sample, therefore when we are forecasting the future, we will need the __forecasts for those features__, because obviously we will not have them at the origin time when creating the future forecast.

Note that it is stil possible to use lag and rolling windows features that have a shorter time frame than the forecasting horizon, but this will mean that we will train a model with say 10 features, but at the origin we will make forecasts using only 5 features, excluding lags and rolling windows. This can have several implications:

- __Out-of-Sample Performance__: The model's performance on the test data (and potentially on out-of-sample data within the testing period) might be reasonably accurate due to the availability of lag and rolling window features during evaluation. However, this performance doesn't guarantee how well the model will generalize to future periods when those features cannot be created.
- __Potential Performance Drop-off__: The model's performance may degrade when forecasting into the future without the lag and rolling window features. This is because the model has learned to rely on those specific features to make predictions, and without them, it might struggle to capture certain patterns or trends.
- __Reduced Feature Space__: In the real-world scenario, if you cannot create lag and rolling window features, you'll be limited to using only the features available at that time (e.g., the 5 features you mentioned). This means the model might not have access to all the information it was trained on, which can limit its forecasting accuracy.

In [ ]:
# Create a copy of the original dataframe
fe_data = 

Create a simple features function and then apply it on our dataset. Use the following [link](https://www.geeksforgeeks.org/python-datetime-datetime-class/) & [link](https://www.geeksforgeeks.org/isocalendar-function-of-datetime-date-class-in-python/) in case you are stuck.

In [ ]:
def extract_dt(df):
    """
    Extracts several datetime objects from a datetime index.
    """
    df['hour'] = df.index.hour                           
    df['dayofweek'] = df.index.dayofweek
    df['quarter'] = df.index.quarter
    df['month'] = df.index.month           # Month
    df['year'] = df.index.          # Year 
    df['dayofyear'] = df.index.dayofyear   # Day of the year
    df['dayofmonth'] = df.       # Day of the month
    df['weekofyear'] =  # Week of the year based on the isocalendaryear

    return df

In [ ]:
# Run the function to create the dataset (input df = fe_data)

fe_data = 

Plot the data.

In [ ]:
plt.figure(figsize=(10, 6))  # Adjust the figure size as needed
plt.plot(fe_data['PJMW_MW'])
plt.xlabel('Index')
plt.ylabel('PJMW_MW')
plt.title('Plot of PJMW_MW')
plt.grid(True)
plt.show()

We could already train and test a model with only these features, as they can be available in the future, because they simply extract datetime information from dates. Instead we are going to add lags as well.

In [8]:
def add_lags(df, target, lags_dict):
    """
    Creates a mapping between index and target that is used to create lags based on
    a dictionary of lag keys and values.
    """
    target_map = df[target].to_dict()
    for lag, lag_days in lags_dict.items():
        df[f'lag_{lag}_year'] = (df.index - pd.Timedelta(lag_days)).map(target_map)
    
    return df

In [10]:
lags_dict = {
    1: "364 days",
    2: "728 days",
    3: "1092 days"
}

fe_data = add_lags(fe_data, target, lags_dict)

Now we need to remove the index, otherwise pycaret will complain. Use the following [link](https://www.w3schools.com/python/pandas/ref_df_reset_index.asp) in case you are stuck.

In [ ]:
# Drop the index of the dataframe 
training_data = fe_data.

# Training and Validation

Remember this is a supervised learning problem, therefore use the [regression module](https://pycaret.readthedocs.io/en/stable/api/regression.html#pycaret.regression.setup) from pycaret.

### Setup 

We want to have names of the features stored as a list. Use the following [link](https://www.geeksforgeeks.org/how-to-drop-one-or-multiple-columns-in-pandas-dataframe/) in case you are stuck.

In [ ]:
# Create a list of the features names and drop the target column from this list 

features = list()
features



We want to create a custom cross-validation fold strategy with a gap.\
[Here](https://medium.com/@soumyachess1496/cross-validation-in-time-series-566ae4981ce4) you can find an explaination of what cross-validation is and how to use it in time-series. \
Please go to the following [link](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.TimeSeriesSplit.html) to find out more about the module you need to use.

In [12]:
# Use this module TimeSeriesSplit with 5 splits and gap =24 (use the link above for guidance)

fold_strategy =

[Link](https://pycaret.readthedocs.io/en/stable/api/regression.html#pycaret.regression.setup)

In [ ]:
modeling_setup_params = {
    "data": training_data,
    "target": target,
    "train_size": 0.7, # Proportion of the dataset to be used for training and validation. Should be between 0.0 and 1.0.
    "numeric_features": features, # in this example all features are numeric. pycaret autoamtically detects type of features, but it can sometimes fail
    "imputation_type": "simple", #The type of imputation to use. Can be either ‘simple’ or ‘iterative’. If None, no imputation of missing values is performed.
    "numeric_imputation": "median", # mean/median do not lead to data leakage; due to a bug in pycaret library, dropping missing values results in a difference between features and target column lengths
    "normalize": True, #When set to True, it transforms the features by scaling them to a given range.
    "normalize_method": "zscore", # Defines the method for scaling.
    "data_split_shuffle": False, # consider time dimension, therefore set to False
    "fold_strategy": fold_strategy, #defined in previous step
    "fold": 5,
    "fold_shuffle": False,
    "log_experiment": True,
    "experiment_name": "Baseline",
    "log_plots": True, 
    # "profile": True, # no need to run it every time
    # "log_profile": True,
    "session_id": 420
}

modeling_setup = setup(
    **modeling_setup_params
)

# Train Model

pycaret offers two main ways of creating, training and evaluating models:
- compare_models(): will train and evaluate every model installed
- create_model(): will train and evaluate a chosen model

Let's try first [create_model()](https://pycaret.readthedocs.io/en/stable/api/regression.html#pycaret.regression.create_model). It will train and evaluate a chosen model, we choose the xgboost algorithm:

In [ ]:
starting_params = {
    'n_estimators': 10,
    'objective': 'reg:squarederror',
    'eval_metric': 'rmse',
}
reg = xgb.XGBRegressor(**starting_params)
model = create_model(reg)

We can evaluate several aspects of the model, from the residuals to validation curves (note, it probably freezes the kernel, use it only when needed):

In [ ]:
evaluate_model(model)
# TODO check which features from the display we want to talk about

1. Residuals distribution
- train set: appear to be normally distributed around zero with relatively low standard deviation; fairly good
- test set: appear to be normally distributed around zero, but with a higher standard deviation compared to residuals on train set; model appears unable to capture all underlying relationships in the data

2. Homoscedasticity:
- train set: given the distirbution of residuals, residuals appear to have a fairly constant variance; good
- test set: residuals are not consistently spread as we move along the x-axis

3. Outliers:
- train/test sets: few outliers, should be removed

Actions to take:
- remove outliers
- tune model using cross-validation

Next, we can interpret the model based on the test/hold-out set using SHAP, to get a better understanding of how each feature impacts the model output:

In [ ]:
interpret_model(model, plot="summary")

#  Setup new model

Setting up a new model with outliers removal and ready for CV hyperparameter tuning:

In [ ]:
modeling_setup_params = {
    "data": training_data,
    "target": target,
    "remove_outliers": True,
    "outliers_threshold": 0.01,
    "train_size": 0.7,
    "numeric_features": features,
    "imputation_type": "simple",
    "numeric_imputation": "median",
    "normalize": True,
    "normalize_method": "zscore",
    "polynomial_features": True,
    "data_split_shuffle": False,
    "fold_strategy": fold_strategy,
    "fold": 5,
    "fold_shuffle": False,
    "log_experiment": True,
    "experiment_name": "Candidate 1",
    "log_plots": True,
    "session_id": 420
}

modeling_setup = setup(
    **modeling_setup_params
)

In [ ]:
starting_params = {
    'n_estimators': 100,
    'objective': 'reg:squarederror',
    'eval_metric': 'rmse',
}
reg = xgb.XGBRegressor(**starting_params)
model = create_model(reg)

The performance of the model created on the hold-out set:

In [1]:
predict_model(model)

# Tune Model

Assume that the trained model is promising. The next thing that can be done is to tune it.
<br>
There are many methods available [here](https://pycaret.readthedocs.io/en/latest/api/regression.html#pycaret.regression.tune_model).
<br>
It turns out that __tune-sklearn__ is a faster way to find optimal hyperparameters and it allows to choose different search algorithms, not just random and grid search.

The trained model from before has the following parameters:

In [ ]:
model

In [ ]:
tuned_model_params = {
    "estimator": model,
    "n_iter": 10,
    "early_stopping": True,
    "optimize": "RMSE",
    "search_library": "tune-sklearn",
    "search_algorithm": "hyperopt",
    "return_train_score": True
}
tuned_model = tune_model(**tuned_model_params)

Now we can check the tuned model performance on the hold-out set

holdout_predictions = predict_model(tuned_model)
holdout_predictions

Let's have a look at the tuned model performance. To choose a different score to plot, refer to [link](https://www.scikit-yb.org/en/latest/api/model_selection/learning_curve.html).

In [ ]:
evaluate_model(tuned_model, plot_kwargs={"scoring": "neg_root_mean_squared_error"})

Feature Importance


In [ ]:
interpret_model(tuned_model)